In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

In [3]:
part = pd.read_pickle(r'C:\Users\a.bagherian\Desktop\working\Diabetics Combined\part2_dm_combined.pkl')
age = pd.read_csv(r'C:\Users\a.bagherian\Desktop\working\Ages\part2_dm_age.csv', index_col='member')

In [ ]:
part

In [ ]:
age

In [ ]:
profile = pd.read_pickle(r'C:\Users\a.bagherian\Desktop\IHIO_share_A\profile.pkl')
profile

1 for male\
0 for female

In [7]:
age['gender'] = profile.set_index('member').loc[age.index, 'gender']

In [ ]:
age

In [9]:
age['age'].describe()

count    245345.000000
mean         50.979488
std          18.444126
min          -6.247904
25%          37.408023
50%          52.723875
75%          64.056072
max         121.757463
Name: age, dtype: float64

In [10]:
# Cleaning age a bit.
age['age'] = np.where(age['age'] < 0, 0, age['age'])
age['age'] = np.where(age['age'] > 100, 100, age['age'])

In [11]:
age['age'].describe()

count    245345.000000
mean         50.982542
std          18.431162
min           0.000000
25%          37.408023
50%          52.723875
75%          64.056072
max         100.000000
Name: age, dtype: float64

In [12]:
n_categories = 20

age['age_cat'] = pd.cut(age['age'], n_categories)

In [13]:
age['age_gender_cat'] = age['age_cat'].astype(str) + ' ' + age['gender'].astype(str)

In [ ]:
age

In [ ]:
n_weights = age['age_gender_cat'].value_counts()
n_weights.sort_index()

In [16]:
na_member = age.loc[age['age_cat'].isna()].index

## Importing non-dm data for sampling

In [ ]:
non_dm = pd.read_pickle(r'C:\Users\a.bagherian\Desktop\IHIO_share_A\prescription\part2.pkl')
non_dm

In [ ]:
non_dm = non_dm.drop(part.index, axis=0)
non_dm

In [19]:
non_dm = non_dm.merge(
    profile[['member', 'birthdate', 'gender']],
    how='left',
    on='member'
)

In [20]:
non_dm['date'] = pd.to_datetime(non_dm['date'])

In [21]:
non_dm['birthdate'] = pd.to_datetime(non_dm['birthdate'])

In [22]:
from statistics import mode


non_dm['age'] = (non_dm['date'] - non_dm['birthdate']) / pd.Timedelta(days=365.2425)
non_dm_ages = non_dm.groupby('member')['age'].apply(mode)

In [23]:
non_dm_ages.to_csv(r'C:\Users\a.bagherian\Desktop\working\Ages\part2_non_dm_age.csv')

In [24]:
# non_dm_ages = pd.read_csv(r'C:\Users\a.bagherian\Desktop\working\Ages\part2_non_dm_age.csv')

In [ ]:
non_dm_ages

## Categorizing non_dm data

In [ ]:
non_dm_ages = non_dm_ages.to_frame()
non_dm_ages = non_dm_ages.merge(
    profile[['member', 'gender']],
    how='left',
    on='member'
)

In [ ]:
non_dm_ages

In [ ]:
# Cleaning age a bit.
non_dm_ages['age'] = np.where(non_dm_ages['age'] < 0, 0, non_dm_ages['age'])
non_dm_ages['age'] = np.where(non_dm_ages['age'] > 100, 100, non_dm_ages['age'])

In [ ]:
non_dm_ages['age'].describe()

count    2.237911e+06
mean     3.108420e+01
std      2.009546e+01
min      0.000000e+00
25%      1.370049e+01
50%      3.084800e+01
75%      4.372164e+01
max      1.000000e+02
Name: age, dtype: float64

In [ ]:
non_dm_ages['age_cat'] = pd.cut(non_dm_ages['age'], n_categories)

In [ ]:
non_dm_ages['age_gender_cat'] = non_dm_ages['age_cat'].astype(str) + ' ' + non_dm_ages['gender'].astype(str)

In [ ]:
non_dm_ages

## Sampling

In [ ]:
n_weights.to_frame()

In [ ]:
non_dm_weights = n_weights.loc[non_dm_ages['age_gender_cat']]

In [ ]:
non_dm_ages['n_weights'] = n_weights.loc[non_dm_ages['age_gender_cat']].values

In [ ]:
non_dm_ages

In [ ]:
def sampler(g, random_state=0):
    n = g['n_weights'].iloc[0]
    return g.sample(n=n, random_state=random_state)

In [ ]:
matched_sample = non_dm_ages.groupby('age_gender_cat', group_keys=False).apply(sampler)

In [ ]:
matched_sample

In [ ]:
matched_sample['age_gender_cat'].value_counts().sort_index()

In [ ]:
matched_sample['age'].describe()

count    240507.000000
mean         48.703301
std          18.130364
min           0.000000
25%          34.607145
50%          50.172146
75%          61.556363
max         100.000000
Name: age, dtype: float64

In [ ]:
matched_sample.to_pickle(r'C:\Users\a.bagherian\Desktop\working\part2_matched_sample.pkl')